In [80]:
import torch
torch.__version__


'2.9.1+cpu'

In [81]:
#!pip install scikit-learn


In [82]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from pathlib import Path

train_dir = Path("../data/split/train")
val_dir   = Path("../data/split/val")


In [83]:
train_transforms = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(0.1, 0.1),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

val_transforms = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])


In [84]:
train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
val_dataset   = datasets.ImageFolder(val_dir,   transform=val_transforms)


In [85]:
print("Classes:", train_dataset.classes)
print("Num classes:", len(train_dataset.classes))
img, label = train_dataset[0]
print(img.shape, label)
print(" number of images " , len(train_dataset))


Classes: ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
Num classes: 6
torch.Size([3, 64, 64]) 0
 number of images  1767


In [86]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=64, shuffle=False)


In [91]:
import torch
from torch import nn

class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),     # 64 -> 32

            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),     # 32 -> 16
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * 16 * 16, 128),  
            nn.ReLU(),
            nn.Dropout(0.5),               
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


In [92]:
num_classes = len(train_dataset.classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SimpleCNN(num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-3)


In [93]:
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=1e-3,
    weight_decay=1e-4
)

from torch.optim.lr_scheduler import ReduceLROnPlateau

scheduler = ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.5,
    patience=2,
    threshold=0.001
)


In [94]:
import time

num_epochs = 20
total_start = time.time()

for epoch in range(num_epochs):
    epoch_start = time.time()

    # ----- TRAIN -----
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

       
        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, preds = torch.max(outputs, 1)
        correct_train += (preds == labels).sum().item()
        total_train += labels.size(0)

    avg_train_loss = running_loss / len(train_loader)
    train_acc = correct_train / total_train

    # ----- VALIDATION -----
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            correct_val += (preds == labels).sum().item()
            total_val += labels.size(0)

    avg_val_loss = val_loss / len(val_loader)
    scheduler.step(avg_val_loss)

    val_acc = correct_val / total_val

    epoch_time = time.time() - epoch_start

    print(
        f"Epoch {epoch+1}/{num_epochs} "
        f"- Train Loss: {avg_train_loss:.4f} - Train Acc: {train_acc:.4f} "
        f"- Val Loss: {avg_val_loss:.4f} - Val Acc: {val_acc:.4f} "
        f"- Time: {epoch_time:.2f} sec"
    )

total_time = time.time() - total_start
print(f"\nTotal training time: {total_time:.2f} sec")


Epoch 1/20 - Train Loss: 1.6885 - Train Acc: 0.2620 - Val Loss: 1.4578 - Val Acc: 0.4350 - Time: 22.70 sec
Epoch 2/20 - Train Loss: 1.4557 - Train Acc: 0.4199 - Val Loss: 1.3269 - Val Acc: 0.4483 - Time: 20.70 sec
Epoch 3/20 - Train Loss: 1.3498 - Train Acc: 0.4697 - Val Loss: 1.3510 - Val Acc: 0.4668 - Time: 20.90 sec
Epoch 4/20 - Train Loss: 1.2584 - Train Acc: 0.5054 - Val Loss: 1.1748 - Val Acc: 0.5570 - Time: 26.27 sec
Epoch 5/20 - Train Loss: 1.2315 - Train Acc: 0.5235 - Val Loss: 1.1215 - Val Acc: 0.5942 - Time: 22.29 sec
Epoch 6/20 - Train Loss: 1.1839 - Train Acc: 0.5642 - Val Loss: 1.1230 - Val Acc: 0.5809 - Time: 19.00 sec
Epoch 7/20 - Train Loss: 1.1442 - Train Acc: 0.5603 - Val Loss: 1.0512 - Val Acc: 0.6074 - Time: 21.32 sec
Epoch 8/20 - Train Loss: 1.1103 - Train Acc: 0.5914 - Val Loss: 1.0963 - Val Acc: 0.5836 - Time: 20.42 sec
Epoch 9/20 - Train Loss: 1.0574 - Train Acc: 0.5976 - Val Loss: 0.9968 - Val Acc: 0.6286 - Time: 17.07 sec
Epoch 10/20 - Train Loss: 1.0544 - Tr